In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def image_registration(img1, img2):
    
    img1_color = cv2.imread(img1)  
    img2_color = cv2.imread(img2)    
 
    img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
    height, width = img2.shape

    orb_detector = cv2.ORB_create(5000)
    kp1, d1 = orb_detector.detectAndCompute(img1, None)
    kp2, d2 = orb_detector.detectAndCompute(img2, None)
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    matches = matcher.match(d1, d2)

    matches.sorted(key = lambda x: x.distance)
    matches = matches[:int(len(matches)*0.9)]
    no_of_matches = len(matches)
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
      p1[i, :] = kp1[matches[i].queryIdx].pt
      p2[i, :] = kp2[matches[i].trainIdx].pt

    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)
    transformed_img = cv2.warpPerspective(img1_color,
                        homography, (width, height))
 
    plt.axis('off')
    plt.imshow(transformed_img,cmap='gray', vmin=0, vmax=255)
    plt.show()

In [5]:
def motionTrackingFunction(img1,img2):
    Iref=img1
    Inext=img2
    Iref=np.array(Iref).astype(np.float32)
    Inext=np.array(Inext).astype(np.float32)
    kernel_x = np.array([[-1., 1.], [-1., 1.]])*.25
    kernel_y = np.array([[-1., -1.], [1., 1.]])*.25
    kernel_t = np.array([[1., 1.], [1., 1.]])*.25
    Iref = Iref / 255. 
    Inext = Inext / 255. 
    Ix=cv2.filter2D(Iref,-1,kernel=kernel_x)
    Iy=cv2.filter2D(Iref,-1,kernel=kernel_y)
    It=cv2.filter2D(Iref,-1,kernel=kernel_t)+cv2.filter2D(Inext,-1,kernel=kernel_x)
    Ix,Iy,It=np.array(Ix),np.array(Iy),np.array(It)
    u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))
    return u

for i in range(0,240,30):
    Iref=cv2.imread('/users/iamns45/Desktop/CV/CV-3/Que-1/frame%d.jpg'%i,cv2.IMREAD_GRAYSCALE)
    Inext=cv2.imread('/users/iamns45/Desktop/CV/CV-3/Que-2/frame%d.jpg'%(i+30),cv2.IMREAD_GRAYSCALE)
    print("Motion function estamite for frame" + str(i) + " frame" + str(i+30) + str(motionTrackingFunction(Iref,Inext)))

Motion function estamite for frame0 frame30[[194.50002  195.5       96.25001  ... 194.00002         inf        inf]
 [194.50002  195.5       96.25001  ... 194.00002         inf        inf]
 [275.06458  275.06458  122.06393  ... 273.65033         inf        inf]
 ...
 [337.5      340.50003  214.08621  ... 192.68735  129.63625   98.818184]
 [335.5      338.50003  238.2941   ... 193.04091  194.45363  124.7726  ]
 [334.00003  336.       473.7652   ...  50.609413  49.466145  45.220573]]
Motion function estamite for frame30 frame60[[247.4874   246.07317  246.78027  ... 197.00002  100.00001  203.00002 ]
 [247.4874   246.07321  246.78027  ... 197.00002  100.00001  203.00002 ]
 [       inf        inf 245.36609  ... 140.00716  200.00002  142.83559 ]
 ...
 [144.19986  142.3025   226.00002  ... 180.56607  133.4075   110.41307 ]
 [107.715935 106.77313  160.51326  ... 202.93887  201.87978  285.50003 ]
 [103.977165 102.63552  145.4648   ... 288.00006  405.87622  287.5     ]]
Motion function estamite 

/var/folders/k0/t2wwzh8n4ng5pxbn25wngqyw0000gn/T/ipykernel_16133/708498194.py:15: RuntimeWarning: divide by zero encountered in divide
  u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))
